tensorrt配合tensorflow工作有两种方式：
- 1、uff,当整个模型完全转换成uff时，所有运算完全可以通过tensorrt来加速。
- 2、tf-trt,这种工作方式下，模型中即使有tensorrt不支持的运算，也可以使用tensorrt加速，这部分运算会由tensorflow代替，而其余部分则会由tensorrt加速。


# 1 uff

具体的流程是把模型转freeze 成pb格式，再接着转换成uff格式。

## 1.1对于tensorflow 1.x而言，保存pb模型如下：

In [7]:
import tensorflow as tf
import os
import numpy as np
import shutil
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers import utils
print(tf.__version__)

1.14.0


In [5]:
def create_model(num_classes = 10):    
    inputs = tf.placeholder(dtype=tf.float32,shape=(None,28,28,1),name='input_node')
    net = slim.conv2d(inputs, 64, 5, 1, scope = 'conv1')  
    net = slim.max_pool2d(net, 2, 2, scope = 'pool1')  
    net = slim.conv2d(net, 128, 5, 1, scope = 'conv2')  
    net = slim.max_pool2d(net, 2, 2, scope = 'pool2')  

    shape = net.get_shape().as_list()
    net = tf.reshape(net, [-1, shape[1] * shape[2] * shape[3]])
    net = slim.fully_connected(net, 128, scope = 'fc1')
    net = slim.fully_connected(net, 128, scope = 'fc2')
    net = slim.fully_connected(net, num_classes, activation_fn = None, scope = 'logits')
    probs = tf.nn.softmax(net,name='myoutputnode')
    return probs


In [10]:
model_out_tf1 = 'tf1_pb'
if os.path.exists(model_out_tf1):
    t=input("The path(%s) is exist,do you want to delete it and remake?(y/n):\n" %(test,)).strip()
    while t not in ['y','n']:
        t = input("your input is invalid,please input again(y/n):\n")
    if t =='y':
        shutil.rmtree(model_out_tf1)
        os.makedirs(model_out_tf1)
    else:
        pass
else:
    print("The test path is not exist! make it !\n")
    os.makedirs(model_out_tf1)

The test path is not exist! make it !



In [17]:
#系统的普通信息和警告信息不打印
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
with tf.Graph().as_default() as g:
    #模型未经训练，模型参数为初始化值
    probs = create_model()
    saver=tf.train.Saver()
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess,model_out_tf1+'/model.ckpt')
        graph = tf.get_default_graph()
        input_graph_def = graph.as_graph_def()
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            ['myoutputnode']
        )
        outgraph=tf.graph_util.remove_training_nodes(output_graph_def)
        for node in outgraph.node:
            print('node:',node.name)
        tf.train.write_graph(outgraph,model_out_tf1,'frozen_model_tf1.pb',as_text=False)
        tf.train.write_graph(outgraph,model_out_tf1,'frozen_model_tf1.pbtxt')

INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.
node: input_node
node: conv1/weights
node: conv1/biases
node: conv1/Conv2D
node: conv1/BiasAdd
node: conv1/Relu
node: pool1/MaxPool
node: conv2/weights
node: conv2/biases
node: conv2/Conv2D
node: conv2/BiasAdd
node: conv2/Relu
node: pool2/MaxPool
node: Reshape/shape
node: Reshape
node: fc1/weights
node: fc1/biases
node: fc1/MatMul
node: fc1/BiasAdd
node: fc1/Relu
node: fc2/weights
node: fc2/biases
node: fc2/MatMul
node: fc2/BiasAdd
node: fc2/Relu
node: logits/weights
node: logits/biases
node: logits/MatMul
node: logits/BiasAdd
node: myoutputnode


## 1.2对于tensorflow 1x 中的keras而言，保存pb模型如下：
更多tensorflow1.x(keras)模型转pb,参考：https://blog.csdn.net/u011119817/article/details/103264080

In [55]:
def process_dataset():
    # Import the data
    (x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # Reshape the data
    NUM_TRAIN = 60000
    NUM_TEST = 10000
    x_train = np.reshape(x_train, (NUM_TRAIN, 28, 28, 1))
    x_test = np.reshape(x_test, (NUM_TEST, 28, 28, 1))
    return x_train, y_train, x_test, y_test

def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[28,28, 1],name="input_node"))
    model.add(tf.keras.layers.Conv2D(64,5,1,activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Conv2D(128,5,1,activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax,name='myoutputnode'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def save(model, filename):
    # First freeze the graph and remove training nodes.
    output_names = model.output.op.name
    sess = tf.keras.backend.get_session()
    frozen_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), [output_names])
    frozen_graph = tf.graph_util.remove_training_nodes(frozen_graph)
    # Save the model
    #tf.train.write_graph(frozen_graph,'tf2_pb','a.pb',as_text=False) #这个也可以
    with open(filename, "wb") as ofile:
        ofile.write(frozen_graph.SerializeToString())

def main(save_path):
    #x_train, y_train, x_test, y_test = process_dataset()
    model = create_model()
    model.summary()
    #不训练也可以
    # Train the model on the data
    #model.fit(x_train, y_train, epochs = 5, verbose = 1)
    # Evaluate the model on test data
    #model.evaluate(x_test, y_test)
    tf.keras.models.save_model(model,'tf1_pb/test.h5') #保存模型后加以加载
    return model

In [45]:
model_out_keras1 = 'tf1_pb'
if os.path.exists(model_out_keras1):
    t=input("The path(%s) is exist,do you want to delete it and remake?(y/n):\n" %(model_out_keras1,)).strip()
    while t not in ['y','n']:
        t = input("your input is invalid,please input again(y/n):\n")
    if t =='y':
        shutil.rmtree(model_out_keras1)
        os.makedirs(model_out_keras1)
    else:
        pass
else:
    print("The test path is not exist! make it !\n")
    os.makedirs(model_out_keras1)

The path(tf1_pb) is exist,do you want to delete it and remake?(y/n):
n


In [56]:
tf.reset_default_graph()
tf.keras.backend.set_learning_phase(0)
tf.keras.backend.clear_session()
hdf5_model=main(model_out_keras1)
def freeze_session(session,keep_var_names=None,output_names=None,clear_devices=True):
    graph = session.graph
    with graph.as_default():
#         freeze_var_names = list(set(v.op.name for v in tf1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
#         output_names += [v.op.name for v in tf1.global_variables()]
        print("output_names",output_names)
        input_graph_def = graph.as_graph_def()
#         for node in input_graph_def.node:
#             print('node:', node.name)
        print("len node1",len(input_graph_def.node))
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph =  tf.graph_util.convert_variables_to_constants(session, input_graph_def,
                                                      output_names)
        
        outgraph = tf.graph_util.remove_training_nodes(frozen_graph)#去掉与推理无关的内容
        print("##################################################################")
        for node in outgraph.node:
            print('node:', node.name)
        print("len node1",len(outgraph.node))
        return outgraph

frozen_graph = freeze_session(tf.keras.backend.get_session(),output_names=[out.op.name for out in hdf5_model.outputs])
tf.train.write_graph(frozen_graph, model_out_keras1, "frozen_model_keras1.pb", as_text=False)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1

'tf1_pb/frozen_model_keras1.pb'

## 1.3 调用两个pb并分析


In [57]:
def load_graph(file_path):

    with tf.gfile.GFile(file_path,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def,input_map = None,return_elements = None,name = "",op_dict = None,producer_op_list = None)
            graph_nodes = [n for n in graph_def.node]
            return graph,graph_nodes

In [58]:
ll tf1_pb

total 23464
-rw-rw-r-- 1 nano1       77 2月  10 15:59 checkpoint
-rw-rw-r-- 1 nano1  1950030 2月  11 10:43 frozen_model_keras1.pb
-rw-rw-r-- 1 nano1  4111847 2月  10 15:59 frozen_model_tf1.pb
-rw-rw-r-- 1 nano1 11837730 2月  10 15:59 frozen_model_tf1.pbtxt
-rw-rw-r-- 1 nano1  4109352 2月  10 15:59 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 nano1      464 2月  10 15:59 model.ckpt.index
-rw-rw-r-- 1 nano1    24294 2月  10 15:59 model.ckpt.meta
-rw-rw-r-- 1 nano1  1970344 2月  11 10:43 test.h5


In [66]:
data=np.random.rand(1,28,28,1)
np.save("tf1_pb/test_data.npy",data)

In [59]:
graph1,graph_nodes1=load_graph('tf1_pb/frozen_model_tf1.pb')
graph2,graph_nodes2=load_graph('tf1_pb/frozen_model_keras1.pb')

In [67]:
input1 = graph1.get_tensor_by_name('input_node:0')
output1 = graph1.get_tensor_by_name('myoutputnode:0')

with tf.Session(graph=graph1) as sess:
    softmax_values = sess.run(output1, feed_dict = {input1: data})
    
    for i, probs in enumerate(softmax_values):
        probs = np.squeeze(probs)
        print(probs)

[0.10509919 0.10234036 0.09939048 0.1059031  0.09565801 0.10008987
 0.10413399 0.09897614 0.10693712 0.08147174]


In [68]:
input2 = graph2.get_tensor_by_name('input_node:0')
output2 = graph2.get_tensor_by_name('myoutputnode/Softmax:0')

with tf.Session(graph=graph2) as sess:
    softmax_values = sess.run(output2, feed_dict = {input2: data})
    
    for i, probs in enumerate(softmax_values):
        probs = np.squeeze(probs)
        print(probs)

[0.09139289 0.10160429 0.1018744  0.11472408 0.10025626 0.09705692
 0.10135855 0.09361386 0.09708725 0.10103158]


# 2 将pb模型转换成uff

In [80]:
import uff
print("uff.version",uff.__version__)

uff.version 0.6.3


In [77]:
def convert(filename,output_node):
    output_filename = filename[:filename.rfind('.')]  + '.uff'
    print('output filename:',output_filename)
    trt_graph = uff.from_tensorflow_frozen_model(filename, output_nodes=[output_node])
    print('Done')
    print('Writing to disk...')
    with open(output_filename, 'wb') as f:
        f.write(trt_graph)
    print('Done')

In [78]:
convert('tf1_pb/frozen_model_tf1.pb','myoutputnode')

output filename: tf1_pb/frozen_model_tf1.uff
NOTE: UFF has been tested with TensorFlow 1.12.0. Other versions are not guaranteed to work
UFF Version 0.6.3
=== Automatically deduced input nodes ===
[name: "input_node"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
}
]

Using output node myoutputnode
Converting to UFF graph
No. nodes: 31
Done
Writing to disk...
Done


In [81]:
convert('tf1_pb/frozen_model_keras1.pb','myoutputnode/Softmax')

output filename: tf1_pb/frozen_model_keras1.uff
NOTE: UFF has been tested with TensorFlow 1.12.0. Other versions are not guaranteed to work
UFF Version 0.6.3
=== Automatically deduced input nodes ===
[name: "input_node"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
}
]

Using output node myoutputnode/Softmax
Converting to UFF graph
DEBUG: convert reshape to flatten node
No. nodes: 30
Done
Writing to disk...
Done


In [82]:
ll tf1_pb/

total 29400
-rw-rw-r-- 1 nano1       77 2月  10 15:59 checkpoint
-rw-rw-r-- 1 nano1  1950030 2月  11 10:43 frozen_model_keras1.pb
-rw-rw-r-- 1 nano1  1949875 2月  11 11:24 frozen_model_keras1.uff
-rw-rw-r-- 1 nano1  4111847 2月  10 15:59 frozen_model_tf1.pb
-rw-rw-r-- 1 nano1 11837730 2月  10 15:59 frozen_model_tf1.pbtxt
-rw-rw-r-- 1 nano1  4112546 2月  11 11:22 frozen_model_tf1.uff
-rw-rw-r-- 1 nano1  4109352 2月  10 15:59 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 nano1      464 2月  10 15:59 model.ckpt.index
-rw-rw-r-- 1 nano1    24294 2月  10 15:59 model.ckpt.meta
-rw-rw-r-- 1 nano1     6400 2月  11 10:50 test_data.npy
-rw-rw-r-- 1 nano1  1970344 2月  11 10:43 test.h5


# 3 调用uff模型
可以重新启动一下，把其它无关进程关闭来测试这个模型效果

In [1]:
from random import randint
from PIL import Image
import numpy as np
import pdb

import pycuda.driver as cuda
# This import causes pycuda to automatically manage CUDA context creation and cleanup.
import pycuda.autoinit

import tensorrt as trt

import sys, os


# You can set the logger severity higher to suppress messages (or lower to display more messages).
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)



#### 运行第一个模型

In [2]:
ll tf1_pb

total 29404
-rw-rw-r-- 1 nano1       77 2月  10 15:59 checkpoint
-rw-rw-r-- 1 nano1  1950030 2月  11 10:43 frozen_model_keras1.pb
-rw-rw-r-- 1 nano1  1949875 2月  11 11:24 frozen_model_keras1.uff
-rw-rw-r-- 1 nano1  4111847 2月  10 15:59 frozen_model_tf1.pb
-rw-rw-r-- 1 nano1 11837730 2月  10 15:59 frozen_model_tf1.pbtxt
-rw-rw-r-- 1 nano1  4112546 2月  11 11:22 frozen_model_tf1.uff
-rw-rw-r-- 1 nano1      617 2月  12 18:32 infer.py
-rw-rw-r-- 1 nano1  4109352 2月  10 15:59 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 nano1      464 2月  10 15:59 model.ckpt.index
-rw-rw-r-- 1 nano1    24294 2月  10 15:59 model.ckpt.meta
-rw-rw-r-- 1 nano1     6400 2月  11 10:50 test_data.npy
-rw-rw-r-- 1 nano1  1970344 2月  11 10:43 test.h5


In [3]:
model_file = 'tf1_pb/frozen_model_tf1.uff'

In [7]:
def GiB(val):
    return val * 1 << 30

class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        print('size:',size)
        print('max_batch_size:',engine.max_batch_size)
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream

def build_engine(model_file):
    # For more information on TRT basics, refer to the introductory samples.
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:
        builder.max_workspace_size = GiB(1)
        # Parse the Uff Network
        parser.register_input(ModelData.INPUT_NAME, ModelData.INPUT_SHAPE)
        parser.register_output(ModelData.OUTPUT_NAME)
        parser.parse(model_file, network)
        # Build and return an engine.
        return builder.build_cuda_engine(network)
# Loads a test case into the provided pagelocked_buffer.
def load_test_case(pagelocked_buffer):
    # Flatten the image into a 1D array, normalize, and copy to pagelocked memory.
    img = np.load('tf1_pb/test_data.npy').ravel()
    np.copyto(pagelocked_buffer, img)
    
# This function is generalized for multiple inputs/outputs.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [5]:
class ModelData(object):
    MODEL_FILE = model_file
    INPUT_NAME ="input_node"
    INPUT_SHAPE = (1, 28, 28)
    OUTPUT_NAME = "myoutputnode"

In [10]:
with build_engine(model_file) as engine:
    # Build an engine, allocate buffers and create a stream.
    # For more information on buffer allocation, refer to the introductory samples.
    #pdb.set_trace()
    inputs, outputs, bindings, stream = allocate_buffers(engine)
    with engine.create_execution_context() as context:
        load_test_case(pagelocked_buffer=inputs[0].host)
        # For more information on performing inference, refer to the introductory samples.
        # The common.do_inference function will return a list of outputs - we only have one in this case.
        [output] = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
        pred = np.argmax(output)
        print("probs:",output)
        print("Prediction: " + str(pred))

size: 784
max_batch_size: 1
size: 10
max_batch_size: 1
probs: [0.10509752 0.10234042 0.09939019 0.10590362 0.09565762 0.10009055
 0.10413482 0.0989755  0.10693821 0.08147158]
Prediction: 8


#### 运行第二个模型

In [11]:
model_file='tf1_pb/frozen_model_keras1.uff'
class ModelData(object):
    MODEL_FILE = model_file
    INPUT_NAME ="input_node"
    INPUT_SHAPE = (1, 28, 28)
    OUTPUT_NAME = "myoutputnode/Softmax"

In [12]:
with build_engine(model_file) as engine:
    # Build an engine, allocate buffers and create a stream.
    # For more information on buffer allocation, refer to the introductory samples.
    #pdb.set_trace()
    inputs, outputs, bindings, stream = allocate_buffers(engine)
    with engine.create_execution_context() as context:
        load_test_case(pagelocked_buffer=inputs[0].host)
        # For more information on performing inference, refer to the introductory samples.
        # The common.do_inference function will return a list of outputs - we only have one in this case.
        [output] = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
        pred = np.argmax(output)
        print("probs:",output)
        print("Prediction: " + str(pred))

size: 784
max_batch_size: 1
size: 10
max_batch_size: 1
probs: [0.09139289 0.10160428 0.1018744  0.11472408 0.10025626 0.09705692
 0.10135854 0.09361386 0.09708725 0.10103157]
Prediction: 3
